In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder, label_binarize, OrdinalEncoder
from sklearn.model_selection import train_test_split


/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [35]:
train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv', index_col='id')
submission = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

In [36]:
train.head()
train.info()

test.head()
test.info()

target = 'Premium Amount'

<class 'pandas.core.frame.DataFrame'>
Index: 1200000 entries, 0 to 1199999
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1181295 non-null  float64
 1   Gender                1200000 non-null  object 
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1181471 non-null  object 
 4   Number of Dependents  1090328 non-null  float64
 5   Education Level       1200000 non-null  object 
 6   Occupation            841925 non-null   object 
 7   Health Score          1125924 non-null  float64
 8   Location              1200000 non-null  object 
 9   Policy Type           1200000 non-null  object 
 10  Previous Claims       835971 non-null   float64
 11  Vehicle Age           1199994 non-null  float64
 12  Credit Score          1062118 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Policy Start Date     1200000 non-null 

In [37]:
# 카테고리 변수에 대해서 제거하지 않고 Unknown으로 지정
cat_col = train.drop(target, axis=1).select_dtypes(include=['object','category'],).columns.tolist()
train[cat_col] = train[cat_col].fillna('Unknown')
test[cat_col] = test[cat_col].fillna('Unknown')

num_col = train.drop(target, axis=1).select_dtypes(exclude=['object','category'],).columns.tolist()
train[num_col] = train[num_col].fillna(train[num_col].mean())
test[num_col] = test[num_col].fillna(test[num_col].mean())


In [38]:
# 이상치 제거
Q1 = train[target].quantile(0.25)
Q3 = train[target].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
train = train[(train[target] >= lower_limit) & (train[target] <= upper_limit)]
train.reset_index(drop=True, inplace=True) 

len(train)

1150680

In [39]:
# 타킷 로그변환
train_y = train[target]
train_y_log = np.log(train[target] + 1) 


In [40]:
# 인코딩
cat_features_card = {}
for f in cat_col:
    cat_features_card[f]=train[f].unique()
cat_features_card

{'Gender': array(['Female', 'Male'], dtype=object),
 'Marital Status': array(['Married', 'Divorced', 'Single', 'Unknown'], dtype=object),
 'Education Level': array(["Bachelor's", "Master's", 'High School', 'PhD'], dtype=object),
 'Occupation': array(['Self-Employed', 'Unknown', 'Employed', 'Unemployed'], dtype=object),
 'Location': array(['Urban', 'Rural', 'Suburban'], dtype=object),
 'Policy Type': array(['Premium', 'Comprehensive', 'Basic'], dtype=object),
 'Policy Start Date': array(['2023-12-23 15:21:39.134960', '2023-06-12 15:21:39.111551',
        '2023-09-30 15:21:39.221386', ..., '2021-04-28 15:21:39.129190',
        '2019-11-14 15:21:39.201446', '2020-10-19 15:21:39.118178'],
       dtype=object),
 'Customer Feedback': array(['Poor', 'Average', 'Good', 'Unknown'], dtype=object),
 'Smoking Status': array(['No', 'Yes'], dtype=object),
 'Exercise Frequency': array(['Weekly', 'Monthly', 'Daily', 'Rarely'], dtype=object),
 'Property Type': array(['House', 'Apartment', 'Condo'], dty

In [41]:
train['Policy Start Date'] = pd.to_datetime(train['Policy Start Date'])
train['Day'] = train['Policy Start Date'].dt.day
train.drop('Policy Start Date', axis=1, inplace=True)

test['Policy Start Date'] = pd.to_datetime(test['Policy Start Date'])
test['Day'] = test['Policy Start Date'].dt.day
test.drop('Policy Start Date', axis=1, inplace=True)

cat_col = train.drop(target, axis=1).select_dtypes(include=['object','category'],).columns.tolist()

df = pd.concat([test, train])

In [42]:
scaler = StandardScaler()
df[num_col] = scaler.fit_transform(df[num_col])

train_enc = df[~df[target].isna()]
test_enc = df[df[target].isna()].drop(target, axis=1)

train_enc = train_enc.drop(target, axis=1) 

In [43]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [44]:
from sklearn.model_selection import train_test_split

cat_features = train_enc.select_dtypes(include=['object']).columns.values

X_train, X_valid, y_train_log, y_valid_log = train_test_split(
    train_enc, 
    train_y_log,
    random_state=101,
    test_size=0.2
)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [46]:
cat_features

array(['Gender', 'Marital Status', 'Education Level', 'Occupation',
       'Location', 'Policy Type', 'Customer Feedback', 'Smoking Status',
       'Exercise Frequency', 'Property Type'], dtype=object)

In [47]:
cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    eval_metric="RMSE",
    random_seed=101,
    verbose=200,
    task_type='GPU',
    l2_leaf_reg=0.7,
)

cat_model.fit(
    X_train, 
    y_train_log,  # 로그 변환된 y
    eval_set=(X_valid, y_valid_log),
    early_stopping_rounds=300,
    cat_features=cat_features,  # 범주형 컬럼 지정
)

cat_pred_log = cat_model.predict(X_valid)

cat_rmsle=rmsle(np.expm1(y_valid_log), np.expm1(cat_pred_log))
print(f"cat_pred RMSLE: {cat_rmsle}")

0:	learn: 1.0634513	test: 1.0667434	best: 1.0667434 (0)	total: 78.1ms	remaining: 1m 18s
200:	learn: 1.0118188	test: 1.0162196	best: 1.0162196 (200)	total: 14.8s	remaining: 58.7s
400:	learn: 1.0092399	test: 1.0157919	best: 1.0157896 (371)	total: 29.4s	remaining: 43.9s
600:	learn: 1.0067651	test: 1.0157067	best: 1.0157005 (597)	total: 44.2s	remaining: 29.3s
800:	learn: 1.0044114	test: 1.0157199	best: 1.0157005 (597)	total: 58.9s	remaining: 14.6s
bestTest = 1.015700509
bestIteration = 597
Shrink model to first 598 iterations.
cat_pred RMSLE: 1.0157005448897996


In [48]:
# 인코딩

custom_categories = [
    ['Female', 'Male'],   
    ['Single', 'Married', 'Divorced', 'Unknown'],
    ['High School', "Bachelor's", "Master's",  'PhD'],
    ['Unemployed', 'Self-Employed',  'Employed', 'Unknown'],
    [ 'Rural', 'Suburban','Urban'],
    ['Basic', 'Comprehensive', 'Premium' ],
    ['Poor', 'Average', 'Good', 'Unknown'],
    ['No', 'Yes'],
    ['Rarely', 'Monthly','Weekly', 'Daily' ],
    ['House', 'Apartment', 'Condo']
]


from sklearn.preprocessing import OrdinalEncoder

X_full = pd.concat([X_train, X_valid], axis=0)

oe = OrdinalEncoder(categories=custom_categories)  # custom_categories는 사전 정의된 범주
X_full[cat_features] = oe.fit_transform(X_full[cat_features]).astype('int')

X_train_enc = X_full.iloc[:len(X_train), :]  # train 영역
X_valid_enc = X_full.iloc[len(X_train):, :]  # valid 영역

In [49]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    random_state=101
)

rf_model.fit(X_train_enc, y_train_log)
rf_pred_log = rf_model.predict(X_valid_enc)
'''
rf_rmsle=rmsle(np.expm1(y_valid_log), np.expm1(rf_pred_log))
print(f"rf_pred RMSLE: {rf_rmsle}")'''

'\nrf_rmsle=rmsle(np.expm1(y_valid_log), np.expm1(rf_pred_log))\nprint(f"rf_pred RMSLE: {rf_rmsle}")'

In [50]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    random_state=101,
    tree_method='gpu_hist',  # GPU 사용
    predictor='gpu_predictor',  # GPU로 예측
    eval_metric='rmse'
)

xgb_model.fit(X_train_enc, y_train_log)
xgb_pred_log = xgb_model.predict(X_valid_enc)
'''xgb_rmsle=rmsle(np.expm1(y_valid_log), np.expm1(xgb_pred_log))
print(f"xgb_pred RMSLE: {xgb_rmsle}")'''

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:33:33] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:33:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:33:37] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:33:37] WARNING: /workspace/src/common/error_ms

'xgb_rmsle=rmsle(np.expm1(y_valid_log), np.expm1(xgb_pred_log))\nprint(f"xgb_pred RMSLE: {xgb_rmsle}")'

In [51]:
cat_pred_log  = cat_model.predict(X_valid)         # CatBoost
rf_pred_log   = rf_model.predict(X_valid_enc)      # RandomForest
xgb_pred_log  = xgb_model.predict(X_valid_enc)     # XGBoost

ensemble_pred_log = (cat_pred_log + rf_pred_log + xgb_pred_log) / 3.0

# 로그 -> 원래 스케일 (expm1)
cat_pred     = np.expm1(cat_pred_log)
rf_pred      = np.expm1(rf_pred_log)
xgb_pred     = np.expm1(xgb_pred_log)
ensemble_pred = np.expm1(ensemble_pred_log)

# y_valid_log도 expm1
y_valid = np.expm1(y_valid_log)

rmsle_cat      = rmsle(y_valid, cat_pred)
rmsle_rf       = rmsle(y_valid, rf_pred)
rmsle_xgb      = rmsle(y_valid, xgb_pred)
rmsle_ensemble = rmsle(y_valid, ensemble_pred)

print(f"RMSLE (CatBoost): {rmsle_cat:.5f}")
print(f"RMSLE (RandomForest): {rmsle_rf:.5f}")
print(f"RMSLE (XGBoost): {rmsle_xgb:.5f}")
print(f"RMSLE (Ensemble): {rmsle_ensemble:.5f}")


RMSLE (CatBoost): 1.01570
RMSLE (RandomForest): 1.01663
RMSLE (XGBoost): 1.01655
RMSLE (Ensemble): 1.01497


In [54]:
cat_pred_log1  = cat_model.predict(test_enc)         # CatBoost

test_enc[cat_features] = oe.fit_transform(test_enc[cat_features]).astype('int')
rf_pred_log1  = rf_model.predict(test_enc)      # RandomForest
xgb_pred_log1  = xgb_model.predict(test_enc)     # XGBoost

ensemble_pred_log1 = (cat_pred_log1 + rf_pred_log1 + xgb_pred_log1) / 3.0

ensemble_pred = np.expm1(ensemble_pred_log1)

In [56]:
test_id = test_enc.index
submission = pd.DataFrame({
    'id': test_id,
    'Premium Amount': ensemble_pred  
})

submission.to_csv('submission.csv', index=False)